In [1]:
import os

import audeer

src_url = '...'
dst_path = 'model.zip'
model_root = 'model'

# audeer.download_url(src_url, dst_path, verbose=True)
audeer.extract_archive(dst_path, model_root, verbose=True)

['model.onnx', 'model.yaml']

In [2]:
import audonnx

model = audonnx.load(model_root)
model

Input:
  signal:
    shape: [1, -1]
    dtype: tensor(float)
    transform: None
Output:
  hidden_states:
    shape: [1, 1024]
    dtype: tensor(float)
    labels: [hidden_states-0, hidden_states-1, hidden_states-2, (...), hidden_states-1021,
      hidden_states-1022, hidden_states-1023]
  logits:
    shape: [1, 3]
    dtype: tensor(float)
    labels: [arousal, dominance, valence]

In [3]:
import gtts

sentences = [
    'Hello world',
    'This is wonderful',
    'This is stupid',
]

files = []
for sentence in audeer.progress_bar(sentences, desc='Create files'):
    file = f'{sentence}.mp3'
    files.append(file)
    if not os.path.exists(file):
        tts = gtts.gTTS(sentence)
        tts.save(file)

In [4]:
import audinterface

interface = audinterface.Feature(
    model.outputs['logits'].labels,
    process_func=model,
    process_func_args={
        'output_names': 'logits',
    },
    sampling_rate=16000,
    resample=True,    
    verbose=True,
)
interface.process_files(files)

,,,arousal,dominance,valence
file,start,end,,,
Hello world.mp3,0 days,0 days 00:00:01.416000,0.503116,0.579775,0.594592
This is wonderful.mp3,0 days,0 days 00:00:01.632000,0.547029,0.647840,0.897838
This is stupid.mp3,0 days,0 days 00:00:01.560000,0.552540,0.659153,-0.002927
